# Imports

In [1]:
!pip install transformers
!pip install datasets
!pip install sentencepiece

In [2]:
import math
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import argparse
import os
import random
import shutil
import time
import torch.nn.parallel
import torch.backends.cudnn as cudnn
from tqdm import tqdm
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.autograd import Variable
from PIL import Image
import torch.utils.data as data
from torchvision.datasets.utils import download_url, check_integrity
import sys
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.preprocessing import StandardScaler
import pickle
from sklearn.gaussian_process.kernels import RBF
import torch
from scipy.stats import multivariate_normal
import  scipy.stats as st
from matplotlib import cm
import torch.optim as optim
from __future__ import print_function
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, AdamW
import nltk
from torch.utils.data import DataLoader
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)


cuda:0


# HotpotQA


In [3]:
DISTRACTORS_PARS_LEN = 0

In [4]:
from datasets import load_dataset

dataset = load_dataset("hotpot_qa", 'distractor')

Reusing dataset hotpot_qa (/root/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/2079f58c1c29624e56735ac2c00efe8808ce52a014e1297552384edae85e3e6d)


In [5]:
train_dataset = dataset['train']
validation_dataset = dataset['validation']

only 6% of the data has yes/no answers, we can  safely remove them

filtering steps:

1.    remove yes-no questions
2.   join sentences together
3. keep only 1 distractor paragraphs 




In [6]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-distilled-squad")
model = AutoModelForQuestionAnswering.from_pretrained("distilbert-base-uncased-distilled-squad")

In [7]:

def get_token_lenght(context, question):
    inputs = tokenizer.encode_plus(question, context, add_special_tokens=True, return_tensors="pt")
    return len(inputs['input_ids'][0])

def remove_distractors(example):
    all_pars = example['context']['title']
    gold_pars = list(set(example['supporting_facts']['title']))
    distractor_pars = list(set(all_pars) - set(gold_pars))
    # get indices to keep from disractors
    if len(distractor_pars) == 0 or DISTRACTORS_PARS_LEN == 0:
        distract_indices = []
    else:
        distract_indices = random.sample(range(len(distractor_pars)), DISTRACTORS_PARS_LEN)
    distractor_pars = [distractor_pars[idx] for idx in distract_indices]
    keep_pars = gold_pars + distractor_pars
    keep_pars_indices = [all_pars.index(keep_par) for keep_par in keep_pars]
    example['context']['title'] = [example['context']['title'][idx] for idx in keep_pars_indices]
    example['context']['sentences'] = ["".join(example['context']['sentences'][idx]) for idx in keep_pars_indices] 

    question = example['question']
    context = "".join(example['context']['sentences'])
    lenght = get_token_lenght(context, question)
    if lenght >= 512:
        all_pars = example['context']['title']
        gold_pars = list(set(example['supporting_facts']['title']))
        # get indices to keep from disractors
        keep_pars = gold_pars
        keep_pars_indices = [all_pars.index(keep_par) for keep_par in keep_pars]
        example['context']['title'] = [example['context']['title'][idx] for idx in keep_pars_indices]
        example['context']['sentences'] = ["".join(example['context']['sentences'][idx]) for idx in keep_pars_indices] 
    return example

In [8]:
train_dataset = train_dataset.map(remove_distractors)
validation_dataset = validation_dataset.map(remove_distractors)

Loading cached processed dataset at /root/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/2079f58c1c29624e56735ac2c00efe8808ce52a014e1297552384edae85e3e6d/cache-920d0aabfc4fbdee.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/hotpot_qa/distractor/1.0.0/2079f58c1c29624e56735ac2c00efe8808ce52a014e1297552384edae85e3e6d/cache-bec4c3da12d92db8.arrow


In [9]:
# remove yes/no questions and oversized questions
indices_train = []
indices_val = []
questions_train = train_dataset[:]['question']
contexts_train = train_dataset[:]['context']
questions_val = validation_dataset[:]['question']
contexts_val = validation_dataset[:]['context']
answers_train = train_dataset[:]['answer']
answers_val = validation_dataset[:]['answer']

for i in range(len(answers_train)):
    if answers_train[i] not in ['yes','no']:
        question = questions_train[i]
        context = "".join(contexts_train[i]['sentences'])
        lenght = get_token_lenght(context, question)
        if lenght <= 500:
            indices_train.append(i)
for i in range(len(answers_val)):
    if answers_val[i] not in ['yes','no']:
        question = questions_val[i]
        context = "".join(contexts_val[i]['sentences'])
        lenght = get_token_lenght(context, question)
        if lenght <= 500:
            indices_val.append(i)


train_dataset = train_dataset.select(indices_train)
validation_dataset = validation_dataset.select(indices_val)

Token indices sequence length is longer than the specified maximum sequence length for this model (594 > 512). Running this sequence through the model will result in indexing errors


In [10]:
train_dataset

Dataset({
    features: ['answer', 'context', 'id', 'level', 'question', 'supporting_facts', 'type'],
    num_rows: 84754
})

In [11]:
validation_dataset

Dataset({
    features: ['answer', 'context', 'id', 'level', 'question', 'supporting_facts', 'type'],
    num_rows: 6935
})

In [ ]:

def remove_oversized_ex(example):
    question = example['question']
    context = "".join(example['context']['sentences'])
    lenght = get_token_lenght(context, question)
    if lenght >= 512:
        all_pars = example['context']['title']
        gold_pars = list(set(example['supporting_facts']['title']))
        # get indices to keep from disractors
        keep_pars = gold_pars
        keep_pars_indices = [all_pars.index(keep_par) for keep_par in keep_pars]
        example['context']['title'] = [example['context']['title'][idx] for idx in keep_pars_indices]
        example['context']['sentences'] = ["".join(example['context']['sentences'][idx]) for idx in keep_pars_indices] 
    return example
    # remove oversized
'''
indices_train = []
indices_val = []
answers_train = train_dataset[:]['answer']
answers_val = validation_dataset[:]['answer']

questions_train = train_dataset[:]['question']
contexts_train = train_dataset[:]['context']
questions_val = validation_dataset[:]['question']
contexts_val = validation_dataset[:]['context']
for i in range(len(train_dataset)):
    question = questions_train[i]
    context = "".join(contexts_train[i]['sentences'])
    lenght = get_token_lenght(context, question)

    if lenght <= 500:
        indices_train.append(i)
for i in range(len(validation_dataset)):
    question = questions_val[i]
    context = "".join(contexts_val[i]['sentences'])
    lenght = get_token_lenght(context, question)
    if lenght <= 500:
        indices_val.append(i)

train_dataset = train_dataset.select(indices_train)
validation_dataset = validation_dataset.select(indices_val)
'''

## token lenghts

In [ ]:
max_token_lenght = 0
lenghts_train = []
lenghts_val = []
exceed_max_train = 0
exceed_max_val = 0
questions_train = train_dataset[:]['question']
contexts_train = train_dataset[:]['context']
questions_val = validation_dataset[:]['question']
contexts_val = validation_dataset[:]['context']
for i in range(len(train_dataset)):
    question = questions_train[i]
    context = "".join(contexts_train[i]['sentences'])
    lenght = get_token_lenght(context, question)
    lenghts_train.append(lenght)
    if lenght >= max_token_lenght:
        max_token_lenght = lenght
    if lenght >= 512:
        exceed_max_train += 1 
    if i % 100:
        print(f'on example {i/len(train_dataset)*100:.3f}')
for i in range(len(validation_dataset)):
    question = questions_val[i]
    context = "".join(contexts_val[i]['sentences'])
    lenght = qa_model.get_token_lenght(context, question)
    lenghts_val.append(lenght)
    if lenght >= max_token_lenght:
        max_token_lenght = lenght
    if lenght >= 512:
        exceed_max_val += 1 
    if i % 100:
        print(f'on example {i/len(validation_dataset)*100:.3f}')
print(f'max token lenght {max_token_lenght} and exceed 512 {exceed_max_val} {exceed_max_train}')

In [ ]:
n, bins, patches = plt.hist(x=lenghts_train, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)
plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Token lenghts in Train')

In [ ]:
n, bins, patches = plt.hist(x=lenghts_val, bins='auto', color='#0504aa',
                            alpha=0.7, rwidth=0.85)


plt.grid(axis='y', alpha=0.75)
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.title('Token lenghts in Val')



In [ ]:
# count yes/no questions in train
total = 0
answers = train_dataset[:]['answer']
for i in range(len(train_dataset)):
    if answers[i] in ['yes','no']:
        total += 1

print(total/len(train_dataset)*100)

# QA model on Hotpot

## squad

In [ ]:
!mkdir squad
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/train-v2.0.json -O squad/train-v2.0.json
!wget https://rajpurkar.github.io/SQuAD-explorer/dataset/dev-v2.0.json -O squad/dev-v2.0.json

In [ ]:
import json
from pathlib import Path

def read_squad(path):
    path = Path(path)
    with open(path, 'rb') as f:
        squad_dict = json.load(f)

    contexts = []
    questions = []
    answers = []
    for group in squad_dict['data']:
        for passage in group['paragraphs']:
            context = passage['context']
            for qa in passage['qas']:
                question = qa['question']
                for answer in qa['answers']:
                    contexts.append(context)
                    questions.append(question)
                    answers.append(answer)

    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_squad('squad/train-v2.0.json')
val_contexts, val_questions, val_answers = read_squad('squad/dev-v2.0.json')

def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters

add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
#train_encodings = tokenizer(train_contexts, train_questions, truncation=True, padding=True)
val_encodings = tokenizer(val_contexts, val_questions, truncation=True, padding=True)
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = tokenizer.model_max_length

        # if end position is None, the 'char_to_token' function points to the space before the correct token - > add + 1
        if end_positions[-1] is None:
            end_positions[-1] = encodings.char_to_token(i, answers[i]['answer_end'] + 1)
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

#add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [ ]:
def add_token_positions(encodings, answers):
    start_positions = []
    end_positions = []
    for i in range(len(answers)):
        is_da = False
        start_positions.append(encodings.char_to_token(i, answers[i]['answer_start']))
        end_positions.append(encodings.char_to_token(i, answers[i]['answer_end']))

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            is_da = True
            start_positions[-1] = tokenizer.model_max_length

        # if end position is None, the 'char_to_token' function points to the space before the correct token - > add + 1
        if end_positions[-1] is None:
            is_da = True
            end_positions[-1] = tokenizer.model_max_length
        if is_da:
            print("sbit")
        else:
            print("gud")
    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

#add_token_positions(train_encodings, train_answers)
add_token_positions(val_encodings, val_answers)

In [ ]:
(train_encodings.char_to_token(0, train_answers[0]['answer_start']))


In [ ]:
train_answers[0]

## hotpot

In [12]:
from transformers import DistilBertTokenizerFast
#tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
tokenizer = AutoTokenizer.from_pretrained("csarron/mobilebert-uncased-squad-v1")

model = AutoModelForQuestionAnswering.from_pretrained("csarron/mobilebert-uncased-squad-v1")

In [13]:
def read_hotpot(hotpot_dataset):

    contexts = []
    questions = []
    answers = []
    all_contexts = hotpot_dataset[:]['context']
    all_questions = hotpot_dataset[:]['question']
    all_answers = hotpot_dataset[:]['answer']
    invalid_quests = 0
    for i in range(len(hotpot_dataset)):

        answer = {}
        context = "".join(all_contexts[i]['sentences'])
        answer['text'] = all_answers[i]
        answer_start = context.find(answer['text'])
        if answer_start == -1:
            invalid_quests += 1
        else:
            answer['answer_start'] = answer_start
            answers.append(answer)
            contexts.append(context)
            questions.append(all_questions[i])
    print(f'invalid percentage {invalid_quests/len(hotpot_dataset)*100:.2f}')
    return contexts, questions, answers

train_contexts, train_questions, train_answers = read_hotpot(train_dataset)
val_contexts, val_questions, val_answers = read_hotpot(validation_dataset)

invalid percentage 0.00
invalid percentage 0.00


In [14]:
def add_end_idx(answers, contexts):
    for answer, context in zip(answers, contexts):
        gold_text = answer['text']
        start_idx = answer['answer_start']
        end_idx = start_idx + len(gold_text)

        # sometimes squad answers are off by a character or two – fix this
        if context[start_idx:end_idx] == gold_text:
            answer['answer_end'] = end_idx
        elif context[start_idx-1:end_idx-1] == gold_text:
            answer['answer_start'] = start_idx - 1
            answer['answer_end'] = end_idx - 1     # When the gold label is off by one character
        elif context[start_idx-2:end_idx-2] == gold_text:
            answer['answer_start'] = start_idx - 2
            answer['answer_end'] = end_idx - 2     # When the gold label is off by two characters
    
add_end_idx(train_answers, train_contexts)
add_end_idx(val_answers, val_contexts)

In [15]:
train_encodings = tokenizer(train_questions, train_contexts,  truncation=True, padding=True, max_length =500, return_tensors="pt")
val_encodings = tokenizer(val_questions, val_contexts, truncation=True, padding=True, max_length =500)
val_ans_encodings = tokenizer([val_answers[i]['text'] for i in range(len(val_answers))],)
train_ans_encodings = tokenizer([train_answers[i]['text'] for i in range(len(train_answers))])

In [51]:
def add_token_positions(encodings, answers):
    def find_answer_in_context(context,answer):
        for j in range((len(context))):
            if context[j:j+len(answer)] == answer:
                return j
        return None     
    start_positions = []
    end_positions = []
    for i in range(len(answers[:])):
        ans = answers[i].tokens[1:-1]
        cntxt = encodings[i].tokens

        start_pos =  find_answer_in_context(cntxt, ans)
        start_positions.append(start_pos)

        # if start position is None, the answer passage has been truncated
        if start_positions[-1] is None:
            start_positions[-1] = 500
            end_positions.append(500)
        else:
            end_positions.append(start_pos + len(ans))

    encodings.update({'start_positions': start_positions, 'end_positions': end_positions})

add_token_positions(train_encodings, train_ans_encodings)
add_token_positions(val_encodings, val_ans_encodings)

In [52]:
import torch

class HotpotDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings.input_ids)
BATCH_SIZE = 8
train_dataset = HotpotDataset(train_encodings)
val_dataset = HotpotDataset(val_encodings)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=False)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [ ]:
del val_dataset
del dataset
del train_dataset
torch.cuda.empty_cache()

In [ ]:
train_encodings['start_positions'][:1000]

In [ ]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

model.to(device)
model.train()

In [ ]:
try:
    i = 0
    for batch in train_loader:
        print(i)
        i += 1
except:
    print(i)
    print(batch)

In [59]:
from transformers import get_linear_schedule_with_warmup
n_epochs = 4
model.train()
optimizer = AdamW(model.parameters(), lr=3e-5)
scheduler = get_linear_schedule_with_warmup(optimizer, 500, len(train_questions)*n_epochs/BATCH_SIZE)

from tqdm import tqdm
my_list = list(range(len(train_questions)*n_epochs))
with tqdm(total=len(my_list)) as pbar:
    for epoch in range(n_epochs):
        print("####################")
        print("New epoch")
        print("####################")
        for batch in train_loader:
            optimizer.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
            answer_start_scores = outputs['start_logits'][0]
            answer_end_scores = outputs['end_logits'][0]
            answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
            answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
            loss = outputs[0]
            loss.backward()
            optimizer.step()
            scheduler.step()
            pbar.update(BATCH_SIZE)
        model.eval()
        model_evaluation(model, tokenizer)
        
#model.eval()

  0%|          | 0/339016 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  


####################
New epoch
####################


  0%|          | 3/6935 [00:00<05:13, 22.08it/s]

context Shirley Temple Black (April 23, 1928 – February 10, 2014) was an American actress, singer, dancer, businesswoman, and diplomat who was Hollywood's number one box-office draw as a child actress from 1935 to 1938. As an adult, she was named United States ambassador to Ghana and to Czechoslovakia and also served as Chief of Protocol of the United States.Kiss and Tell is a 1945 American comedy film starring then 17-year-old Shirley Temple as Corliss Archer. In the film, two teenage girls cause their respective parents much concern when they start to become interested in boys. The parents' bickering about which girl is the worse influence causes more problems than it solves.
quest What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?
truth Chief of Protocol
pred united states ambassador to



  1%|▏         | 104/6935 [00:04<05:22, 21.19it/s]

...

context Boules was featured in the Summer Olympic Games unofficial programme in 1900.The Summer Olympic Games (French: "Jeux olympiques d'été" ) or the Games of the Olympiad, first held in 1896, is an international multi-sport event that is hosted by a different city every four years. The most recent Olympics were held in Rio de Janeiro, Brazil. The International Olympic Committee organizes the games and oversees the host city's preparations. In each Olympic event, gold medals are awarded for first place, silver medals are awarded for second place, and bronze medals are awarded for third; this tradition began in 1904. The Winter Olympic Games were created due to the success of the Summer Olympics.
quest Which Olympic game featured Boules and happened right before medals started to be awarded?
truth The Summer Olympic Games
pred summer olympic games unofficial



100%|█████████▉| 6904/6935 [07:12<00:01, 15.55it/s]

context Daniel Vacek (born 1 April 1971) is a former tennis player from Czechoslovakia and the Czech Republic who turned professional in 1990. He reached the quarterfinals of the 1995 Paris Masters, the 1998 Canada Masters and the 1998 Cincinnati Masters, and achieved a career-high singles ranking of World No. 26 in January 1996.Fabio Fognini (] ; born 24 May 1987) is an Italian professional tennis player who is currently ranked world No. 29 in men's singles by the Association of Tennis Professionals (ATP) and the current Italian No. 1. His career-high singles ranking is world No. 13, achieved in March 2014, and world No. 7 in doubles, achieved in July 2015. Fognini's most successful surface is red clay, upon which he won his four ATP singles titles in Stuttgart, Hamburg, Viña del Mar and Umag, reached the quarterfinals of the 2011 French Open and the semifinals of the 2013 Monte-Carlo Masters. Together with Simone Bolelli, Fognini won the 2015 Australian Open doubles event, becoming t


100%|██████████| 6935/6935 [07:14<00:00, 15.97it/s]
339040it [4:31:16, 20.83it/s]

{'exact_match': 34.67916366258111, 'f1': 67.03954999253327}


339040it [4:31:16, 20.83it/s]{'exact_match': 34.67916366258111, 'f1': 67.03954999253327}


In [56]:
def get_answer( model, tokenizer, context, question):
    # 1. TOKENIZE THE INPUT
    # note: if you don't include return_tensors='pt' you'll get a list of lists which is easier for
    # exploration but you cannot feed that into a model.
    inputs = tokenizer.encode_plus(question, context, return_tensors="pt")
    inputs = inputs.to(device)
    # 2. OBTAIN MODEL SCORES
    # the AutoModelForQuestionAnswering class includes a span predictor on top of the model.
    # the model returns answer start and end scores for each word in the text
    answer_start_scores, answer_end_scores = model(**inputs, return_dict=False)
    answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
    answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
    # 3. GET THE ANSWER SPAN
    # once we have the most likely start and end tokens, we grab all the tokens between them
    # and convert tokens back to words!
    answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(inputs["input_ids"][0][answer_start:answer_end]))
    return answer
# https://huggingface.co/transformers/migration.html
def model_evaluation(model, tokenizer):
    preds = []
    my_list = list(range(len(val_questions)))
    with tqdm(total=len(my_list)) as pbar:
        for ex in range(len(val_questions)):
            answer = get_answer(model, tokenizer, val_contexts[ex],val_questions[ex])
            preds.append(answer)
            if ex % 100 == 0:
                print("context " +val_contexts[ex] )
                print("quest " +val_questions[ex] )
                print("truth " +val_answers[ex]['text'] )
                print("pred " + answer)
            pbar.update(1)
    truths = [val_answers[i]['text'] for i in range(len(val_answers))]
    scores = evaluate_truth_pred(truths, preds)
    print(scores)
    return scores

In [55]:
from __future__ import print_function
from collections import Counter
import string
import re
import argparse
import json
import sys


def normalize_answer(s):
    """Lower text and remove punctuation, articles and extra whitespace."""
    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))


def f1_score(prediction, ground_truth):
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return f1


def exact_match_score(prediction, ground_truth):
    return (normalize_answer(prediction) == normalize_answer(ground_truth))


def metric_max_over_ground_truths(metric_fn, prediction, ground_truths):
    scores_for_ground_truths = []
    for ground_truth in ground_truths:
        score = metric_fn(prediction, ground_truth)
        scores_for_ground_truths.append(score)
    return max(scores_for_ground_truths)



def evaluate_truth_pred(truths, preds):
    '''
    truths, preds: matched arrays of ground truth answers and predictions
    '''
    f1 = exact_match = total = 0
    for i in range(len(truths)):
        total += 1
        ground_truths = [truths[i]]
        prediction = preds[i]
        exact_match += metric_max_over_ground_truths(
            exact_match_score, prediction, ground_truths)
        f1 += metric_max_over_ground_truths(
            f1_score, prediction, ground_truths)

    exact_match = 100.0 * exact_match / total
    f1 = 100.0 * f1 / total

    return {'exact_match': exact_match, 'f1': f1}


In [57]:
model.eval()
model_evaluation(model, tokenizer)

  0%|          | 5/6935 [00:00<05:54, 19.56it/s]

context Shirley Temple Black (April 23, 1928 – February 10, 2014) was an American actress, singer, dancer, businesswoman, and diplomat who was Hollywood's number one box-office draw as a child actress from 1935 to 1938. As an adult, she was named United States ambassador to Ghana and to Czechoslovakia and also served as Chief of Protocol of the United States.Kiss and Tell is a 1945 American comedy film starring then 17-year-old Shirley Temple as Corliss Archer. In the film, two teenage girls cause their respective parents much concern when they start to become interested in boys. The parents' bickering about which girl is the worse influence causes more problems than it solves.
quest What government position was held by the woman who portrayed Corliss Archer in the film Kiss and Tell?
truth Chief of Protocol
pred united states ambassador to ghana and to czechoslovakia and also served as chief of protocol of the united states.


  1%|▏         | 103/6935 [00:04<05:03, 22.53it/s]

context George Archainbaud (May 7, 1890 – February 20, 1959) was a French-born American film and television director.Ralph Murphy (May 1, 1895 – February 10, 1967) was an American film director. Born in Rockville, Connecticut, Murphy was active in films from 1931 through 1962, with some work in television. From 1941–44 he was married to Gloria Dickson, whom he directed in "I Want a Divorce".
quest Who died first, George Archainbaud or Ralph Murphy?
truth George Archainbaud
pred george archainbaud or


  3%|▎         | 205/6935 [00:09<05:12, 21.53it/s]

...

100%|█████████▉| 6903/6935 [05:16<00:01, 21.80it/s]

context Daniel Vacek (born 1 April 1971) is a former tennis player from Czechoslovakia and the Czech Republic who turned professional in 1990. He reached the quarterfinals of the 1995 Paris Masters, the 1998 Canada Masters and the 1998 Cincinnati Masters, and achieved a career-high singles ranking of World No. 26 in January 1996.Fabio Fognini (] ; born 24 May 1987) is an Italian professional tennis player who is currently ranked world No. 29 in men's singles by the Association of Tennis Professionals (ATP) and the current Italian No. 1. His career-high singles ranking is world No. 13, achieved in March 2014, and world No. 7 in doubles, achieved in July 2015. Fognini's most successful surface is red clay, upon which he won his four ATP singles titles in Stuttgart, Hamburg, Viña del Mar and Umag, reached the quarterfinals of the 2011 French Open and the semifinals of the 2013 Monte-Carlo Masters. Together with Simone Bolelli, Fognini won the 2015 Australian Open doubles event, becoming t

100%|██████████| 6935/6935 [05:17<00:00, 21.81it/s]


{'exact_match': 33.15068493150685, 'f1': 64.25937672646107}


{'exact_match': 33.15068493150685, 'f1': 64.25937672646107}

{'exact_match': 33.15068493150685, 'f1': 64.25937672646107}


In [ ]:
model.eval()
model_evaluation(model, tokenizer)

pre-trained: {'exact_match': 35.039653929343906, 'f1': 49.006594520884434}


trained with 1 distractor: {'exact_match': 35.93772733886222, 'f1': 48.90843722013081}


In [ ]:

with tqdm(total=len(my_list)) as pbar:
    for epoch in range(1):
        for batch in train_loader:
            optim.zero_grad()
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            start_positions = batch['start_positions'].to(device)
            end_positions = batch['end_positions'].to(device)
            outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
            answer_start_scores = outputs['start_logits'][0]
            answer_end_scores = outputs['end_logits'][0]
            answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
            answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
            loss = outputs[0]
            loss.backward()
            print(loss)
            optim.step()
            pbar.update(BATCH_SIZE)

#model.eval()

In [ ]:
input_ids = batch['input_ids'].to(device)
attention_mask = batch['attention_mask'].to(device)
start_positions = batch['start_positions'].to(device)
end_positions = batch['end_positions'].to(device)
outputs = model(input_ids, attention_mask=attention_mask, start_positions=start_positions, end_positions=end_positions)
answer_start_scores = outputs['start_logits'][2]
answer_end_scores = outputs['end_logits'][2]
answer_start = torch.argmax(answer_start_scores)  # get the most likely beginning of answer with the argmax of the score
answer_end = torch.argmax(answer_end_scores) + 1  # get the most likely end of answer with the argmax of the score
answer = tokenizer.convert_tokens_to_string(tokenizer.convert_ids_to_tokens(input_ids[2][answer_start:answer_end]))
print(answer)

In [ ]:
model.eval()
preds = []
my_list = list(range(len(val_questions)))
with tqdm(total=len(my_list)) as pbar:
    for ex in range(len(val_questions[:10])):
        answer = get_answer(model, tokenizer, val_contexts[ex],val_questions[ex])
        preds.append(answer)
        pbar.update(1)
truths = [val_answers[i]['text'] for i in range(len(val_answers[:10]))]
scores = evaluate_truth_pred(truths, preds)
print(scores)


- getting embeddings from model
- set cover strategies
- code for deferall with general expert and AI